<a href="https://colab.research.google.com/github/kmenesesrivera/codonusagebias/blob/main/2-Comparaci%C3%B3n_de_Modelos_Selecci%C3%B3n_de_Caracteristicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Comparación de Modelos  

###Lectura de la Data

In [3]:
import urllib.request
import zipfile, urllib.request, shutil
from __future__ import absolute_import, division, print_function, unicode_literals

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00577/codon_usage.csv.zip'
filename = 'codon_usage.csv.zip'

In [4]:
with urllib.request.urlopen(url) as response, open(filename, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)
    with zipfile.ZipFile(filename) as zf:
        zf.extractall()

In [5]:
!unzip codon_usage.csv

In [20]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import tensorflow.python.keras, math
from sklearn.model_selection import train_test_split
labelencoder = LabelEncoder()

In [11]:
#Lectura del dataset
file_name = "codon_usage.csv"
dataset =  pd.read_csv('https://drive.google.com/uc?export=download&id=1Z4v43cvTwp920NyOdboDKP7_ytC_0tBC')

# Se observa que hay datos str que no permiten manipular los demás como numéricos.
dataset[['UUU', 'UUC']] = dataset[['UUU', 'UUC']].apply(pd.to_numeric, errors='coerce')
null_UUU = dataset['UUU'].isna().sum()
null_UUC= dataset['UUC'].isna().sum()
print ("Cantidad de datos nulos en codon UUU ",null_UUU)
print ("Cantidad de datos nuls en codon UUC ",null_UUC)



Cantidad de datos nulos en codon UUU  2
Cantidad de datos nuls en codon UUC  1


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
def preprocess_dataset(dataset, save_metadata=True):
  
  preprocessed_dataset = dataset.copy()

  #Determinamos aquellas variables que tengan un porcentaje de registros únicos por cada variable mayor al valor de 0.7
  threshold = 0.7

  unique_percentages = dataset.nunique() / len(dataset)

  criteria = unique_percentages > threshold

  columns_to_filter = unique_percentages[criteria].keys()
  
  # Está columna contiene datos descriptivos, por tanto será transformada a str.

  preprocessed_dataset['SpeciesName'] = preprocessed_dataset['SpeciesName'].astype(str)
  preprocessed_dataset['SpeciesName']
 
  #Eliminamos las variables seleccionadas en el paso anterior
  preprocessed_dataset.drop(columns_to_filter, axis=1, inplace=True)
 
  #Eliminamos los registros duplicados
  preprocessed_dataset.drop_duplicates(keep="first", inplace=True)

  #Determinamos que variables son del tipo númerico y cuales son categóricas
  numeric_columns = list()
  categorical_columns = list()
  dictionary_of_columns_with_index_to_categorical = dict()
  dictionary_of_columns_with_categorical_to_index = dict()

  for column in preprocessed_dataset:
    #Determinamos si la variable es numérica o no
    if pd.api.types.is_numeric_dtype(preprocessed_dataset[column]):
      numeric_columns.append(column)
    else:
      #Modificamos el tipo de dato de la variable mediante "astype"
      preprocessed_dataset[column] = preprocessed_dataset[column].astype("category")

      #Verificamos si el tipo de dato de la variable fue transformado a categórico correctamente
      if not pd.api.types.is_categorical_dtype(preprocessed_dataset[column]):
        raise Exception("La columna {} no se transformó correctamente a categórica".format(column))

      dictionary_of_columns_with_index_to_categorical[column] = dict()
      dictionary_of_columns_with_categorical_to_index[column] = dict()
      
      #Indexamos los valores (categorías), sin tomar en consideración los nulos, de la variable y guardamos esa información en los diccionarios
      for index, category in enumerate(preprocessed_dataset[column].cat.categories):
        dictionary_of_columns_with_index_to_categorical[column][index] = category
        dictionary_of_columns_with_categorical_to_index[column][category] = index
      
      categorical_columns.append(column)
  
  #Reemplazamos los nulos con la mediana sólo de aquellas variables numéricas
    median_of_numeric_columns = preprocessed_dataset[numeric_columns].median()
    preprocessed_dataset[numeric_columns] = preprocessed_dataset[numeric_columns].fillna(median_of_numeric_columns)

  #Transformamos a números los valores (categorías) de las variables categóricas sin considerar los nulos
  preprocessed_dataset.replace(dictionary_of_columns_with_categorical_to_index, inplace=True)

  #Determinamos aquellas variables que tengan un porcentaje de registros nulos por cada variable mayor al valor de 0.7
  threshold = 0.7

  null_percentages = preprocessed_dataset[categorical_columns].isna().sum() / len(preprocessed_dataset)

  criteria = null_percentages > threshold

  columns_to_filter = null_percentages[criteria].keys()

  #Eliminamos las variables seleccionadas en el paso anterior
  preprocessed_dataset.drop(columns_to_filter, axis=1, inplace=True)

  #Eliminamos los registros duplicados
  preprocessed_dataset.drop_duplicates(keep="first", inplace=True)
 
  return preprocessed_dataset

In [13]:
preprocessed_dataset = preprocess_dataset(dataset)


In [14]:
preprocessed_dataset.describe

<bound method NDFrame.describe of        Kingdom  DNAtype   Ncodons      UUU  ...      GAG      UAA      UAG      UGA
0            9        0      1995  0.01654  ...  0.04361  0.00251  0.00050  0.00000
1            9        0      1474  0.02714  ...  0.04410  0.00271  0.00068  0.00000
2            9        0      4862  0.01974  ...  0.02468  0.00391  0.00000  0.00144
3            9        0      1915  0.01775  ...  0.03446  0.00261  0.00157  0.00000
4            9        0     22831  0.02816  ...  0.03679  0.00000  0.00044  0.00131
...        ...      ...       ...      ...  ...      ...      ...      ...      ...
13023        7        0      1097  0.02552  ...  0.04102  0.00091  0.00091  0.00638
13024        7        1      2067  0.01258  ...  0.00677  0.00242  0.00097  0.01887
13025        7        1      1686  0.01423  ...  0.00297  0.00356  0.00119  0.02017
13026        7        0  40662582  0.01757  ...  0.03959  0.00099  0.00079  0.00156
13027        7        1   8998998  0.01778

In [15]:
#Separamos las columnas  Reino y ADN  , respectivamente.
X1 = preprocessed_dataset.drop("Kingdom", axis=1)
X2 = preprocessed_dataset.drop("DNAtype", axis=1)
Y1 = preprocessed_dataset["Kingdom"]  
Y2 = preprocessed_dataset["DNAtype"] 

In [16]:
#Mediante el método "train_test_split" usaremos el 20% de la data para probar el modelo. El parámetro "random state" nos sirve para
#poder replicar la misma separación
x_train, Xt, y_train, Yt = train_test_split(X1, Y1, test_size=0.2, random_state=70)

In [17]:
#Mediante el método "train_test_split" usaremos el 20% de la data para probar el modelo. El parámetro "random state" nos sirve para
#poder replicar la misma separación
x2_train, Xt2, y2_train, Yt2 = train_test_split(X2, Y2, test_size=0.2, random_state=70)

Ahora vamos a probar los modelos sin tunearlos

##Decision Tree

**Y1**

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(x_train, y_train) #Entrenamos un Decision Tree tunear

In [ ]:
f1_score(Yt, dt.predict(Xt), average='weighted') #Usamos F1 para comparar con otros modelos

**Y2**

In [ ]:
dt2 = DecisionTreeClassifier()
dt2.fit(x2_train, y2_train) #Entrenamos un Decision Tree tunear

In [ ]:
f1_score(Yt2, dt2.predict(Xt2), average='weighted') #Usamos F1 para comparar con otros modelos

##Random Forest

**Y1**

In [ ]:
rf = RandomForestClassifier(40, n_jobs=-1, oob_score=True)
rf.fit(x_train, y_train) #Entrenamos un Random Forest sin tunear

In [ ]:
f1_score(Yt, rf.predict(Xt), average='weighted') #Usamos F1 para comparar con otros modelos

**Y2**

In [ ]:
rf2 = RandomForestClassifier(40, n_jobs=-1, oob_score=True)
rf2.fit(x2_train, y2_train) #Entrenamos un Random Forest sin tunear

In [ ]:
f1_score(Yt2, rf2.predict(Xt2), average='weighted') #Usamos F1 para comparar con otros modelos

##Clasificador K-Nearest Neighbor

**Y1**

In [ ]:
n_neighbors = 4

knn = KNeighborsClassifier(n_neighbors)
knn.fit(x_train, y_train) #Entrenamos un KNN sin tunear

In [ ]:
f1_score(Yt, knn.predict(Xt), average='weighted') #Usamos F1 para comparar con otros modelos

**Y2**

In [31]:
n_neighbors = 4

knn2 = KNeighborsClassifier(n_neighbors)
knn2.fit(x2_train, y2_train) #Entrenamos un KNN sin tunear

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [32]:
f1_score(Yt2, knn2.predict(Xt2), average='weighted') #Usamos F1 para comparar con otros modelos

0.7578392501379395

# Clasificador XGBoost

**Y1**

In [26]:
 xgb=XGBClassifier(n_estimators=80,  max_depth=10)
 xgb.fit(x_train, y_train) #Entrenamos un XGBost sin tunear

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=80, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [28]:
f1_score(Yt, xgb.predict(Xt), average='weighted') #Usamos F1 para comparar con otros modelos

0.9017597020186107

**Y2**

In [29]:
xgb2=XGBClassifier(n_estimators=80, max_depth=10 )
xgb2.fit(x2_train, y2_train) #Entrenamos un XGBost sin tunear


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=80, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [33]:
f1_score(Yt2, xgb2.predict(Xt2), average='weighted') #Usamos F1 para comparar con otros modelos 

0.9916912067241496

# Clasificador LightGBM

**Y1**

In [34]:
mdl = lgb.LGBMClassifier(boosting_type='gbdt',objective='binary')
mdl.fit(x_train, y_train) #Entrenamos un LightGBM sin tunear

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [35]:
f1_score(Yt, mdl.predict(Xt), average='weighted') #Usamos F1 para comparar con otros modelos

0.9080295712276717

**Y2**

In [36]:
mdl2 = lgb.LGBMClassifier(boosting_type='gbdt',objective='binary',max_depth = 10)
mdl2.fit(x2_train, y2_train) #Entrenamos un XGBost sin tunear

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=10,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [37]:
f1_score(Yt2, mdl2.predict(Xt2), average='weighted') #Usamos F1 para comparar con otros modelos

0.9540231327684141

# Redes neuronales

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import tensorflow.python.keras, math
from sklearn.model_selection import train_test_split

In [ ]:
lr = 0.01
bs = 256
epochs = 30
#x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


In [ ]:
nn = Sequential([
    Dense(12, activation='softmax', input_shape=(66,))
])

nn.compile(optimizer=SGD(lr), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
nn.summary()

In [ ]:
nn.fit(x_train, y_train, batch_size=bs, epochs=epochs, validation_data=(Xt, Yt)) #Entrenamos un NN

In [ ]:
xt_predict = np.argmax(nn.predict(Xt), axis=1)
f1_score(Yt, xt_predict, average='weighted')

##Seleccion de Caracteristicas

###Por Filtrado

In [39]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, mutual_info_classif

In [40]:
#Implementamos una filtrado usando `mutual_info_classif` no podemos usar chi2 porque tenemos valores negatiivos
filtrado = SelectKBest(mutual_info_classif, k=5).fit(x_train, y_train)

In [ ]:
filtrado.scores_

In [ ]:
X_new = filtrado.transform(x_train)
X_new[:5] #Vemos nuestro set de featuers filtrado

**Implementamos un bucle por cada modelo y vemos los resutlados de usar las k caracteristicas**

####Decision Tree

In [42]:
max = 0
features = 0
for n in range(66):
  filter = SelectKBest(mutual_info_classif, k=(n+1)).fit(x_train, y_train)
  X_new = filter.transform(x_train)
  Xt_new = filter.transform(Xt)
  dt2 = DecisionTreeClassifier()
  dt2.fit(X_new, y_train)
  acc = dt2.score(Xt_new, Yt)
  f1 = f1_score(Yt, dt2.predict(Xt_new), average='weighted')
  print('Score usando ', (n+1), 'caracteristicas F1:',  f1, ' acurracy: ', acc )
  if f1 > max:
      max = f1
      features = n+1
print('Mejor Score usando ', features, 'caracteristicas F1:',  max)

KeyboardInterrupt: ignored

####Random Forest

In [ ]:
max = 0
features = 0
for n in range(66):
  filter = SelectKBest(mutual_info_classif, k=(n+1)).fit(x_train, y_train)
  X_new = filter.transform(x_train)
  Xt_new = filter.transform(Xt)
  rt2 = RandomForestClassifier(40, n_jobs=-1, oob_score=True)
  rt2.fit(X_new, y_train)
  acc = rt2.score(Xt_new, Yt)
  f1 = f1_score(Yt, rt2.predict(Xt_new), average='weighted')
  print('Score usando ', (n+1), 'caracteristicas F1:',  f1, ' acurracy: ', acc )
  if f1 > max:
      max = f1
      features = n+1
print('Mejor Score usando ', features, 'caracteristicas F1:',  max)

####KNN

In [ ]:
max = 0
features = 0
for n in range(66):
  filter = SelectKBest(mutual_info_classif, k=(n+1)).fit(x_train, y_train)
  X_new = filter.transform(x_train)
  Xt_new = filter.transform(Xt)
  n_neighbors = 4
  knn2 = KNeighborsClassifier(n_neighbors)
  knn2.fit(X_new, y_train)
  acc = knn2.score(Xt_new, Yt)
  f1 = f1_score(Yt, knn2.predict(Xt_new), average='weighted')
  print('Score usando ', (n+1), 'caracteristicas F1:',  f1, ' acurracy: ', acc )
  if f1 > max:
      max = f1
      features = n+1
print('Mejor Score usando ', features, 'caracteristicas F1:',  max)


####XGBoost

In [43]:
max = 0
features = 0
for n in range(66):
  filter = SelectKBest(mutual_info_classif, k=(n+1)).fit(x_train, y_train)
  X_new = filter.transform(x_train)
  Xt_new = filter.transform(Xt)
  xgb=XGBClassifier(n_estimators=80,  max_depth=10)
  xgb.fit(X_new, y_train)
  acc = xgb.score(Xt_new, Yt)
  f1 = f1_score(Yt, xgb.predict(Xt_new), average='weighted')
  print('Score usando ', (n+1), 'caracteristicas F1:',  f1, ' acurracy: ', acc )
  if f1 > max:
      max = f1
      features = n+1
print('Mejor Score usando ', features, 'caracteristicas F1:',  max)


Score usando  1 caracteristicas F1: 0.3946353904322388  acurracy:  0.42978068487879956
Score usando  2 caracteristicas F1: 0.4929752346812626  acurracy:  0.5309734513274337
Score usando  3 caracteristicas F1: 0.5785070502323121  acurracy:  0.5936898807233552
Score usando  4 caracteristicas F1: 0.6284509938998512  acurracy:  0.639091958445556
Score usando  5 caracteristicas F1: 0.6512560667730193  acurracy:  0.6629472874182378
Score usando  6 caracteristicas F1: 0.6782442868498604  acurracy:  0.6898807233551366
Score usando  7 caracteristicas F1: 0.7197973462079876  acurracy:  0.7287418237783763
Score usando  8 caracteristicas F1: 0.7502451869583681  acurracy:  0.7599076567910735
Score usando  9 caracteristicas F1: 0.7587800508095801  acurracy:  0.767602924201616
Score usando  10 caracteristicas F1: 0.7844655067605677  acurracy:  0.7922277799153521
Score usando  11 caracteristicas F1: 0.7847895801588735  acurracy:  0.7922277799153521
Score usando  12 caracteristicas F1: 0.79983360650110

In [ ]:
max = 0
features = 0
for n in range(66):
  filter = SelectKBest(mutual_info_classif, k=(n+1)).fit(x2_train, y2_train)
  X_new = filter.transform(x2_train)
  Xt_new = filter.transform(Xt2)
  xgb=XGBClassifier(n_estimators=80,  max_depth=10)
  xgb.fit(X_new, y2_train)
  acc = xgb.score(Xt_new, Yt2)
  f1 = f1_score(Yt2, xgb.predict(Xt_new), average='weighted')
  print('Score usando ', (n+1), 'caracteristicas F1:',  f1, ' acurracy: ', acc )
  if f1 > max:
      max = f1
      features = n+1
print('Mejor Score usando ', features, 'caracteristicas F1:',  max)


Score usando  1 caracteristicas F1: 0.011042266829243587  acurracy:  0.009619084263178146
Score usando  2 caracteristicas F1: 0.029351178848975673  acurracy:  0.027318199307425933
Score usando  3 caracteristicas F1: 0.01642228473653459  acurracy:  0.012312427856868027
Score usando  4 caracteristicas F1: 0.016966862930026784  acurracy:  0.012312427856868027
Score usando  5 caracteristicas F1: 0.011369858651137927  acurracy:  0.0061562139284340135
Score usando  6 caracteristicas F1: 0.010046976540139425  acurracy:  0.0061562139284340135
Score usando  7 caracteristicas F1: 0.009887937684112795  acurracy:  0.005386687187379761
Score usando  8 caracteristicas F1: 0.011081428139828526  acurracy:  0.005771450557906887
Score usando  9 caracteristicas F1: 0.011488684663825788  acurracy:  0.0061562139284340135
Score usando  10 caracteristicas F1: 0.01130138096142649  acurracy:  0.006540977298961139
Score usando  11 caracteristicas F1: 0.01742291426654321  acurracy:  0.009619084263178146
Score us

#### LightGBM

In [44]:
max = 0
features = 0
for n in range(65):
  filter = SelectKBest(mutual_info_classif, k=(n+1)).fit(x_train, y_train)
  X_new = filter.transform(x_train)
  Xt_new = filter.transform(Xt)
  mdl = lgb.LGBMClassifier(boosting_type='gbdt',objective='binary')
  mdl.fit(X_new, y2_train)
  acc = mdl.score(Xt_new, Yt)
  f1 = f1_score(Yt, mdl.predict(Xt_new), average='weighted')
  print('Score usando ', (n+1), 'caracteristicas F1:',  f1, ' acurracy: ', acc )
  if f1 > max:
      max = f1
      features = n+1
print('Mejor Score usando ', features, 'caracteristicas F1:',  max)


Score usando  1 caracteristicas F1: 0.3806535927060132  acurracy:  0.4301654482493267
Score usando  2 caracteristicas F1: 0.4889782239576172  acurracy:  0.5348210850327049
Score usando  3 caracteristicas F1: 0.5793761982507387  acurracy:  0.596383224317045
Score usando  4 caracteristicas F1: 0.6227335528524948  acurracy:  0.6344747979992305
Score usando  5 caracteristicas F1: 0.6493469977118139  acurracy:  0.6637168141592921
Score usando  6 caracteristicas F1: 0.6740949487502526  acurracy:  0.687956906502501
Score usando  7 caracteristicas F1: 0.7161199202318782  acurracy:  0.7264332435552135
Score usando  8 caracteristicas F1: 0.7476184794390129  acurracy:  0.7587533666794921
Score usando  9 caracteristicas F1: 0.7528251097999803  acurracy:  0.7626010003847634
Score usando  10 caracteristicas F1: 0.7613874849194902  acurracy:  0.7664486340900346
Score usando  11 caracteristicas F1: 0.7744920673984824  acurracy:  0.7783762985763756
Score usando  12 caracteristicas F1: 0.795670983653452

In [ ]:
max = 0
features = 0
for n in range(65):
  filter = SelectKBest(mutual_info_classif, k=(n+1)).fit(x2_train, y2_train)
  X_new = filter.transform(x2_train)
  Xt_new = filter.transform(Xt)
  mdl = lgb.LGBMClassifier(boosting_type='gbdt',objective='binary')
  mdl.fit(X_new, y2_train)
  acc = mdl.score(Xt_new, Yt)
  f1 = f1_score(Yt, mdl.predict(Xt_new), average='weighted')
  print('Score usando ', (n+1), 'caracteristicas F1:',  f1, ' acurracy: ', acc )
  if f1 > max:
      max = f1
      features = n+1
print('Mejor Score usando ', features, 'caracteristicas F1:',  max)

####NN

In [ ]:
max = 0
features = 0
lr = 0.01
bs = 256
epochs = 30
X = x_train
Y = y_train
for n in range(66):
  filter = SelectKBest(mutual_info_classif, k=(n+1)).fit(X, Y)
  X_new = filter.transform(X)
  Xt_new = filter.transform(Xt)
  x_train, x_val, y_train, y_val = train_test_split(X_new, Y, test_size=0.2, random_state=42)

  nn2 = Sequential([
      Dense(12, activation='softmax', input_shape=((n+1),))
  ])

  nn2.compile(optimizer=SGD(lr), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  nn2.fit(x_train, y_train, batch_size=bs, epochs=epochs, validation_data=(x_val, y_val), verbose=0 )
  xt_predict = np.argmax(nn2.predict(Xt_new), axis=1)

  f1 = f1_score(Yt, xt_predict, average='weighted')
  print('Score usando ', (n+1), 'caracteristicas F1:',  f1)
  if f1 > max:
      max = f1
      features = n+1
print('Mejor Score usando ', features, 'caracteristicas F1:',  max)

###Por wrapping - Backward elimination

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

**Apilcamos a cada modelo**

####Decision Tree

In [ ]:
model = DecisionTreeClassifier()
rfecv = RFECV(estimator=model, step=1, cv=StratifiedKFold(2),scoring='accuracy')
rfecv.fit(X, Y1)
plt.figure()
plt.xlabel("Number of features selected")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
rfecv.n_features_

In [ ]:
X_new = rfecv.transform(X)
Xt_new = rfecv.transform(Xt)
dt3 = DecisionTreeClassifier()
dt3.fit(X_new, Y1)
acc = dt3.score(Xt_new, Yt)
f1 = f1_score(Yt, dt3.predict(Xt_new), average='weighted')
print('F1 Score: ', f1)

####Random Forest

In [ ]:
model = RandomForestClassifier(40, n_jobs=-1, oob_score=True)
rfecv = RFECV(estimator=model, step=1, cv=StratifiedKFold(2),scoring='accuracy')
rfecv.fit(X, Y1)
plt.figure()
plt.xlabel("Number of features selected")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
rfecv.n_features_

In [ ]:
X_new = rfecv.transform(X)
Xt_new = rfecv.transform(Xt)
rf3 = RandomForestClassifier(40, n_jobs=-1, oob_score=True)
rf3.fit(X_new, Y1)
acc = rf3.score(Xt_new, Yt)
f1 = f1_score(Yt, rf3.predict(Xt_new), average='weighted')
print('F1 Score: ', f1)

#### XGBoost

In [ ]:
xgb=XGBClassifier(n_estimators=80,  max_depth=10)
rfecv = RFECV(estimator=xgb, step=1, cv=4,scoring='accuracy')
rfecv.fit(X, Y1)
plt.figure()
plt.xlabel("Number of features selected")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
rfecv.n_features_

In [ ]:
X_new = rfecv.transform(X)
Xt_new = rfecv.transform(Xt)
xgb = XGBClassifier(n_estimators=80,  max_depth=10)
xgb.fit(X_new, Y1)
acc = xgb.score(Xt_new, Yt)
f1 = f1_score(Yt, xgb.predict(Xt_new), average='weighted')
print('F1 Score: ', f1)

####LightGBM

In [ ]:
model = LGBMClassifier(boosting_type='gbdt',objective='binary')
rfecv = RFECV(estimator=model, step=1, cv=4,scoring='accuracy')
rfecv.fit(X, Y1)
plt.figure()
plt.xlabel("Number of features selected")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
rfecv.n_features_

In [ ]:
X_new = rfecv.transform(X)
Xt_new = rfecv.transform(Xt)
model = LGBMClassifier(boosting_type='gbdt',objective='binary')
model.fit(X_new, Y1)
acc = model.score(Xt_new, Yt)
f1 = f1_score(Yt, model.predict(Xt_new), average='weighted')
print('F1 Score: ', f1)

####KNN y Neural Network

KNN no expone la importancia de los features asi que no se puede analizar con RFECV y en las Neural Network tambien es complicado extraer esa informacion

###PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [ ]:
pca = PCA().fit(X)
pd.DataFrame(pca.components_, columns=X.columns).head()

In [ ]:
per_var = np.round(pca.explained_variance_ratio_*100, decimals=5)
labels = ['PC' + str(x) for x in range(1, len(per_var) + 1)]
plt.plot(labels, np.cumsum(pca.explained_variance_ratio_), '-s')
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]
#plt.rcParams['figure.figsize'] = 5, 60
plt.show()

In [ ]:
plt.bar(x=range(1, len(per_var) + 1), height=per_var, tick_label = labels)
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]
plt.show()

####Decision Tree

In [ ]:
max = 0
components = 0
for n in range(66):
  pca = PCA(n_components=(n+1))
  X_new = pca.fit_transform(X)
  Xt_new = pca.transform(Xt)
  dt4 = DecisionTreeClassifier()
  dt4.fit(X_new, Y1)
  acc = dt4.score(Xt_new, Yt)
  f1 = f1_score(Yt, dt4.predict(Xt_new), average='weighted')
  print('Score usando ', (n+1), 'componentes F1:',  f1, ' acurracy: ', acc )
  if f1 > max:
      max = f1
      components = n+1
print('Mejor Score usando ', components, 'componentes F1:',  max)

####Random Forest

In [ ]:
max = 0
components = 0
for n in range(66):
  pca = PCA(n_components=(n+1))
  X_new = pca.fit_transform(X)
  Xt_new = pca.transform(Xt)
  rf4 = RandomForestClassifier(40, n_jobs=-1, oob_score=True)
  rf4.fit(X_new, Y1)
  acc = rf4.score(Xt_new, Yt)
  f1 = f1_score(Yt, rf4.predict(Xt_new), average='weighted')
  print('Score usando ', (n+1), 'componentes F1:',  f1, ' acurracy: ', acc )
  if f1 > max:
      max = f1
      components = n+1
print('Mejor Score usando ', components, 'componentes F1:',  max)

####XGBoost

In [ ]:
max = 0
components = 0
for n in range(66):
  pca = PCA(n_components=(n+1))
  X_new = pca.fit_transform(X)
  Xt_new = pca.transform(Xt)
  model = XGBClassifier(n_estimators=80,  max_depth=10)
  model.fit(X_new, Y1)
  acc = model.score(Xt_new, Yt)
  f1 = f1_score(Yt, model.predict(Xt_new), average='weighted')
  print('Score usando ', (n+1), 'componentes F1:',  f1, ' acurracy: ', acc )
  if f1 > max:
      max = f1
      components = n+1
print('Mejor Score usando ', components, 'componentes F1:',  max)

####Light GBM

In [ ]:
max = 0
components = 0
for n in range(66):
  pca = PCA(n_components=(n+1))
  X_new = pca.fit_transform(X)
  Xt_new = pca.transform(Xt)
  model = LGBMClassifier(boosting_type='gbdt',objective='binary')
  model.fit(X_new, Y1)
  acc = model.score(Xt_new, Yt)
  f1 = f1_score(Yt, model.predict(Xt_new), average='weighted')
  print('Score usando ', (n+1), 'componentes F1:',  f1, ' acurracy: ', acc )
  if f1 > max:
      max = f1
      components = n+1
print('Mejor Score usando ', components, 'componentes F1:',  max)

####KNN

In [ ]:
max = 0
components = 0
for n in range(66):
  pca = PCA(n_components=(n+1))
  X_new = pca.fit_transform(X)
  Xt_new = pca.transform(Xt)
  knn4 = KNeighborsClassifier(4)
  knn4.fit(X_new, Y1)
  acc = knn4.score(Xt_new, Yt)
  f1 = f1_score(Yt, knn4.predict(Xt_new), average='weighted')
  print('Score usando ', (n+1), 'componentes F1:',  f1, ' acurracy: ', acc )
  if f1 > max:
      max = f1
      components = n+1
print('Mejor Score usando ', components, 'componentes F1:',  max)